In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import gc
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, LabelBinarizer, OrdinalEncoder
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('latihkredit2.csv')
data.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,83.7,9,0,26,verified
1,2500,60 months,15.27,0.0,RENT,30000.0,car,GA,1.00,9.4,4,1,12,verified
2,2400,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,98.5,10,0,10,not verified
3,10000,36 months,13.49,10.0,RENT,49200.0,other,CA,20.00,21.0,37,0,15,verified
4,5000,36 months,7.90,3.0,RENT,36000.0,wedding,AZ,11.20,28.3,12,0,7,verified


#  Handling Missing Value 

In [11]:
imut = SimpleImputer(missing_values=np.nan, strategy = 'mean')
#num_data.values[:, 3:4] = imut.fit_transform(num_data.values[:, 3:4])
#num_data
data.iloc[:, 3:4] = imut.fit_transform(data.iloc[:, 3:4])
data.isnull().sum().any()


False

# Pembantu 

In [12]:
# Divide category and number
cat_data = []
num_data = []
for i,c in enumerate(data.dtypes):
    if c == object:
        cat_data.append(data.iloc[:, i])
    else :
        num_data.append(data.iloc[:, i])  
cat_data = pd.DataFrame(cat_data).transpose()
num_data = pd.DataFrame(num_data).transpose()

# Melakukan hal yg Diperlukan:

In [13]:
target = cat_data['verification_status']
data.drop('addr_state', axis=1, inplace=True)
cat_data.drop('verification_status', axis=1, inplace=True)
target = target.map({'verified': 1 , 'not verified' : 0})

data.drop('verification_status', axis=1, inplace=True)

#data.dtypes()

In [21]:
data.dtypes


loan_amnt                  int64
term                      object
int_rate                 float64
emp_length               float64
home_ownership            object
annual_inc               float64
purpose                   object
dti                      float64
revol_util               float64
total_acc                  int64
bad_loan                   int64
longest_credit_length      int64
dtype: object

In [22]:
#onehot onecode + standar scaler

col_transformer = ColumnTransformer(
                    transformers=[
                        ("ohe", OneHotEncoder(sparse=False), ['term','home_ownership','purpose']),
                        ("Ss", StandardScaler(), ['loan_amnt','int_rate','emp_length','annual_inc','dti','revol_util','total_acc','bad_loan','longest_credit_length'])
                    ],
                    remainder='passthrough',
                    n_jobs=-1
                    )

featureon = col_transformer.fit_transform(data)

featureon = pd.DataFrame(featureon)
featureon


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,-0.925846,-0.481139,1.361216,-1.033056,2.107808,0.922106,-1.183218,-0.472845,2.000901
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,-1.248307,0.663596,-1.546759,-0.879309,-2.112504,-1.943286,-1.658922,2.114859,-0.156683
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.261205,0.834563,1.361216,-1.334093,-0.889959,1.492870,-1.088077,-0.472845,-0.464909
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.280925,0.222551,1.361216,-0.387319,0.896349,-1.495930,1.480727,-0.472845,0.305657
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-0.925846,-1.162530,-0.674366,-0.725562,-0.497225,-1.214404,-0.897795,-0.472845,-0.927248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,2.943682,-0.042572,-0.674366,1.555018,-0.639750,-0.377540,1.290446,2.114859,0.459770
2995,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.280925,-0.664495,0.000000,-1.094555,-0.660336,-1.013865,-0.517231,-0.472845,2.309127
2996,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,-0.796862,-0.664495,1.361216,-0.725562,-0.544733,-1.792882,2.907841,-0.472845,0.459770
2997,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.961317,-0.042572,-0.092772,-0.687125,1.792670,-1.225974,-0.041527,-0.472845,-0.773135


In [23]:
from sklearn.preprocessing import MinMaxScaler
#onehot encode + min max scaler


col_transformer1 = ColumnTransformer(
                    transformers=[
                        ("ohe", OneHotEncoder(sparse=False), ['term','home_ownership','purpose']),
                        ("Ss", MinMaxScaler(), ['loan_amnt','int_rate','emp_length','annual_inc','dti','revol_util','total_acc','bad_loan','longest_credit_length'])
                    ],
                    remainder='passthrough',
                    n_jobs=-1
                    )

featureonMnMx = col_transformer1.fit_transform(data)

featureonMnMx = pd.DataFrame(featureonMnMx)
featureonMnMx


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.117647,0.256764,1.000000,0.028804,0.926298,0.838677,0.078947,0.0,0.500000
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.044118,0.511872,0.000000,0.039882,0.033501,0.094188,0.013158,1.0,0.195652
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.041176,0.549972,1.000000,0.007112,0.292127,0.986974,0.092105,0.0,0.152174
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.264706,0.413584,1.000000,0.075332,0.670017,0.210421,0.447368,0.0,0.260870
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.117647,0.104914,0.300000,0.050960,0.375209,0.283567,0.118421,0.0,0.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.000000,0.354500,0.300000,0.215288,0.345059,0.501002,0.421053,1.0,0.282609
2995,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.264706,0.215903,0.531902,0.024372,0.340704,0.335671,0.171053,0.0,0.543478
2996,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.147059,0.215903,1.000000,0.050960,0.365159,0.133267,0.644737,0.0,0.282609
2997,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.109559,0.354500,0.500000,0.053730,0.859631,0.280561,0.236842,0.0,0.108696


In [24]:
#ordinal_encoder + standarscaler

col_transformer2 = ColumnTransformer(
                    transformers=[
                        ("OrE", OrdinalEncoder(), ['term','home_ownership','purpose']),
                        ("Ss", StandardScaler(), ['loan_amnt','int_rate','emp_length','annual_inc','dti','revol_util','total_acc','bad_loan','longest_credit_length'])
                    ],
                    remainder='passthrough',
                    n_jobs=-1
                    )

featurore = col_transformer2.fit_transform(data)
featurore = pd.DataFrame(featurore)
featurore


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,2.0,1.0,-0.925846,-0.481139,1.361216,-1.033056,2.107808,0.922106,-1.183218,-0.472845,2.000901
1,1.0,2.0,0.0,-1.248307,0.663596,-1.546759,-0.879309,-2.112504,-1.943286,-1.658922,2.114859,-0.156683
2,0.0,2.0,10.0,-1.261205,0.834563,1.361216,-1.334093,-0.889959,1.492870,-1.088077,-0.472845,-0.464909
3,0.0,2.0,8.0,-0.280925,0.222551,1.361216,-0.387319,0.896349,-1.495930,1.480727,-0.472845,0.305657
4,0.0,2.0,12.0,-0.925846,-1.162530,-0.674366,-0.725562,-0.497225,-1.214404,-0.897795,-0.472845,-0.927248
...,...,...,...,...,...,...,...,...,...,...,...,...
2994,1.0,0.0,2.0,2.943682,-0.042572,-0.674366,1.555018,-0.639750,-0.377540,1.290446,2.114859,0.459770
2995,0.0,2.0,8.0,-0.280925,-0.664495,0.000000,-1.094555,-0.660336,-1.013865,-0.517231,-0.472845,2.309127
2996,0.0,0.0,2.0,-0.796862,-0.664495,1.361216,-0.725562,-0.544733,-1.792882,2.907841,-0.472845,0.459770
2997,1.0,1.0,10.0,-0.961317,-0.042572,-0.092772,-0.687125,1.792670,-1.225974,-0.041527,-0.472845,-0.773135


In [25]:
#ordinal_encoder + MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

col_transformer3 = ColumnTransformer(
                    transformers=[
                        ("OrE", OrdinalEncoder(), ['term','home_ownership','purpose']),
                        ("MnMx", MinMaxScaler(feature_range = (0, 1)), ['loan_amnt','int_rate','emp_length','annual_inc','dti','revol_util','total_acc','bad_loan','longest_credit_length'])
                    ],
                    remainder='passthrough',
                    n_jobs=-1
                    )

featuroreMnMx = col_transformer3.fit_transform(data)
featuroreMnMx = pd.DataFrame(featuroreMnMx)
featuroreMnMx


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,2.0,1.0,0.117647,0.256764,1.000000,0.028804,0.926298,0.838677,0.078947,0.0,0.500000
1,1.0,2.0,0.0,0.044118,0.511872,0.000000,0.039882,0.033501,0.094188,0.013158,1.0,0.195652
2,0.0,2.0,10.0,0.041176,0.549972,1.000000,0.007112,0.292127,0.986974,0.092105,0.0,0.152174
3,0.0,2.0,8.0,0.264706,0.413584,1.000000,0.075332,0.670017,0.210421,0.447368,0.0,0.260870
4,0.0,2.0,12.0,0.117647,0.104914,0.300000,0.050960,0.375209,0.283567,0.118421,0.0,0.086957
...,...,...,...,...,...,...,...,...,...,...,...,...
2994,1.0,0.0,2.0,1.000000,0.354500,0.300000,0.215288,0.345059,0.501002,0.421053,1.0,0.282609
2995,0.0,2.0,8.0,0.264706,0.215903,0.531902,0.024372,0.340704,0.335671,0.171053,0.0,0.543478
2996,0.0,0.0,2.0,0.147059,0.215903,1.000000,0.050960,0.365159,0.133267,0.644737,0.0,0.282609
2997,1.0,1.0,10.0,0.109559,0.354500,0.500000,0.053730,0.859631,0.280561,0.236842,0.0,0.108696


In [52]:

featuroreMnMx

loan_amnt                float64
int_rate                 float64
emp_length               float64
annual_inc               float64
dti                      float64
revol_util               float64
total_acc                float64
bad_loan                 float64
longest_credit_length    float64
dtype: object

In [33]:
X = featuroreMnMx
Y = target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.3)


# Proses Classifict & Confussion Matrices

#  

# Gaussian NB

In [29]:
#Training ordinal+standardscaler
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression(random_state = 0)
classifierLR.fit(X_train, y_train)
# KNearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)
# SVM GAUSSIAN
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'rbf', random_state = 0)
classifierSVM.fit(X_train, y_train)
#Naive bayes
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB(var_smoothing=1e-09)
classifierNB.fit(X_train, y_train)

#predict & confusion matrix ordinal+standardscaler
# Prediction y_pred
y_pred_LR = classifierLR.predict(X_test)
y_pred_KNN = classifierKNN.predict(X_test)
y_pred_SVM = classifierSVM.predict(X_test)
y_pred_NB = classifierNB.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm_logreg = confusion_matrix(y_test, y_pred_LR)
as_logreg=accuracy_score(y_test, y_pred_LR)

cm_knn = confusion_matrix(y_test, y_pred_KNN)
as_knn=accuracy_score(y_test, y_pred_KNN)

cm_svm_gaussian = confusion_matrix(y_test, y_pred_SVM)
as_svm_gaussian = accuracy_score(y_test, y_pred_SVM)

cm_nb = confusion_matrix(y_test, y_pred_NB)
as_nb = accuracy_score(y_test, y_pred_NB)

# Find best Classifier
score={'as_logreg':as_logreg, 'as_knn':as_knn, 'as_svm_gaussian':as_svm_gaussian, 'as_nb':as_nb}
score_list=[]
for i in score:
    score_list.append(score[i])
    u=max(score_list)
    if score[i]==u:
        v=i  
    print(f"{i}={score[i]}");   
print(f"The best method to use in this case is {v} with accuracy score {u}")
print('ini udah ordinal')

as_logreg=0.66
as_knn=0.6455555555555555
as_svm_gaussian=0.6766666666666666
as_nb=0.6233333333333333
The best method to use in this case is as_svm_gaussian with accuracy score 0.6766666666666666
ini udah ordinal


#  

# Gaussian NB

In [31]:
#datainput featurore
# Training Onehotencode+standard_scaler

from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression(random_state = 0)
classifierLR.fit(X_train, y_train)
# KNearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)
# SVM GAUSSIAN
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'rbf', random_state = 0)
classifierSVM.fit(X_train, y_train)
#Naive bayes
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)

#predict & confusion matrix Onehotencode+standard_scaler
# Prediction y_pred
y_pred_LR = classifierLR.predict(X_test)
y_pred_KNN = classifierKNN.predict(X_test)
y_pred_SVM = classifierSVM.predict(X_test)
y_pred_NB = classifierNB.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm_logreg = confusion_matrix(y_test, y_pred_LR)
as_logreg=accuracy_score(y_test, y_pred_LR)

cm_knn = confusion_matrix(y_test, y_pred_KNN)
as_knn=accuracy_score(y_test, y_pred_KNN)

cm_svm_gaussian = confusion_matrix(y_test, y_pred_SVM)
as_svm_gaussian = accuracy_score(y_test, y_pred_SVM)

cm_nb = confusion_matrix(y_test, y_pred_NB)
as_nb = accuracy_score(y_test, y_pred_NB)

# Find best Classifier
score={'as_logreg':as_logreg, 'as_knn':as_knn, 'as_svm_gaussian':as_svm_gaussian, 'as_nb':as_nb}
score_list=[]
for i in score:
    score_list.append(score[i])
    u=max(score_list)
    if score[i]==u:
        v=i  
    print(f"{i}={score[i]}");   
print(f"The best method to use in this case is {v} with accuracy score {u}")
print('apa lagi kalo curse dimension, ato ngga di ordinal?')

as_logreg=0.6988888888888889
as_knn=0.6622222222222223
as_svm_gaussian=0.7111111111111111
as_nb=0.6433333333333333
The best method to use in this case is as_svm_gaussian with accuracy score 0.7111111111111111
apa lagi kalo curse dimension, ato ngga di ordinal?


#  

In [35]:
# data input featuroreMnMx
# Training Onehotencode+standard_scaler 

from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression(random_state = 0)
classifierLR.fit(X_train, y_train)
# KNearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)
# SVM GAUSSIAN
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'rbf', random_state = 0)
classifierSVM.fit(X_train, y_train)
#Naive bayes
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)

#predict & confusion matrix Onehotencode+standard_scaler
# Prediction y_pred
y_pred_LR = classifierLR.predict(X_test)
y_pred_KNN = classifierKNN.predict(X_test)
y_pred_SVM = classifierSVM.predict(X_test)
y_pred_NB = classifierNB.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm_logreg = confusion_matrix(y_test, y_pred_LR)
as_logreg=accuracy_score(y_test, y_pred_LR)

cm_knn = confusion_matrix(y_test, y_pred_KNN)
as_knn=accuracy_score(y_test, y_pred_KNN)

cm_svm_gaussian = confusion_matrix(y_test, y_pred_SVM)
as_svm_gaussian = accuracy_score(y_test, y_pred_SVM)

cm_nb = confusion_matrix(y_test, y_pred_NB)
as_nb = accuracy_score(y_test, y_pred_NB)

# Find best Classifier
score={'as_logreg':as_logreg, 'as_knn':as_knn, 'as_svm_gaussian':as_svm_gaussian, 'as_nb':as_nb}
score_list=[]
for i in score:
    score_list.append(score[i])
    u=max(score_list)
    if score[i]==u:
        v=i  
    print(f"{i}={score[i]}");   
print(f"The best method to use in this case is {v} with accuracy score {u}")
print('apa lagi kalo curse dimension, ato ngga di ordinal?')

as_logreg=0.6722222222222223
as_knn=0.6244444444444445
as_svm_gaussian=0.6277777777777778
as_nb=0.6377777777777778
The best method to use in this case is as_logreg with accuracy score 0.6722222222222223
apa lagi kalo curse dimension, ato ngga di ordinal?


# BernoulliNB

In [36]:
#Ordinalencode+standard_scaler Training
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression(random_state = 0)
classifierLR.fit(X_train, y_train)
# KNearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)
# SVM GAUSSIAN
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'rbf', random_state = 0)
classifierSVM.fit(X_train, y_train)
#Naive bayes
from sklearn.naive_bayes import BernoulliNB
classifierNB = BernoulliNB()
classifierNB.fit(X_train, y_train)

#Ordinalencode+standard_scaler classifict metrics
# Prediction y_pred
y_pred_LR = classifierLR.predict(X_test)
y_pred_KNN = classifierKNN.predict(X_test)
y_pred_SVM = classifierSVM.predict(X_test)
y_pred_NB = classifierNB.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm_logreg = confusion_matrix(y_test, y_pred_LR)
as_logreg=accuracy_score(y_test, y_pred_LR)

cm_knn = confusion_matrix(y_test, y_pred_KNN)
as_knn=accuracy_score(y_test, y_pred_KNN)

cm_svm_gaussian = confusion_matrix(y_test, y_pred_SVM)
as_svm_gaussian = accuracy_score(y_test, y_pred_SVM)

cm_nb = confusion_matrix(y_test, y_pred_NB)
as_nb = accuracy_score(y_test, y_pred_NB)

# Find best Classifier
score={'as_logreg':as_logreg, 'as_knn':as_knn, 'as_svm_gaussian':as_svm_gaussian, 'as_nb':as_nb}
score_list=[]
for i in score:
    score_list.append(score[i])
    u=max(score_list)
    if score[i]==u:
        v=i  
    print(f"{i}={score[i]}");   
print(f"The best method to use in this case is {v} with accuracy score {u}")


as_logreg=0.6722222222222223
as_knn=0.6244444444444445
as_svm_gaussian=0.6277777777777778
as_nb=0.6411111111111111
The best method to use in this case is as_logreg with accuracy score 0.6722222222222223


In [84]:
from sklearn.naive_bayes import MultinomialNB

multiNB = MultinomialNB()

multiNB.fit(X_train, y_train)
print(multiNB)

y_expect = y_test
y_pred = multiNB.predict(X_test)
#print accuracy_score(y_expect, y_pred)


MultinomialNB()


In [ ]:
cm_nb = confusion_matrix(y_test, y_pred_NB)
as_nb = accuracy_score(y_test, y_pred_NB)